# Deep Learning - Assignment 12

# Rachit Aggarwal - M12506500

## Importing Tensor Flow

In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\rachi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Initializing placeholders for input and output

In [4]:
n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10
n_layers = 3

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

## Basic LSTM algorithm

In [3]:
lstm_cells = [tf.contrib.rnn.BasicLSTMCell(num_units=n_neurons)
              for layer in range(n_layers)]
multi_cell = tf.contrib.rnn.MultiRNNCell(lstm_cells)
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)
top_layer_h_state = states[-1][1]
logits = tf.layers.dense(top_layer_h_state, n_outputs, name="softmax")
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")

## Optimizing and evaluating

In [4]:
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

## Initializing variables

In [5]:
init = tf.global_variables_initializer()

## Importing test and train data

In [6]:
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print("Epoch", epoch, "Train accuracy =", acc_train, "Test accuracy =", acc_test)

Epoch 0 Train accuracy = 0.9533333 Test accuracy = 0.939
Epoch 1 Train accuracy = 0.99333334 Test accuracy = 0.9714
Epoch 2 Train accuracy = 0.9866667 Test accuracy = 0.972
Epoch 3 Train accuracy = 0.9866667 Test accuracy = 0.979
Epoch 4 Train accuracy = 0.99333334 Test accuracy = 0.9827
Epoch 5 Train accuracy = 0.99333334 Test accuracy = 0.9806
Epoch 6 Train accuracy = 1.0 Test accuracy = 0.9871
Epoch 7 Train accuracy = 0.99333334 Test accuracy = 0.9874
Epoch 8 Train accuracy = 1.0 Test accuracy = 0.9869
Epoch 9 Train accuracy = 0.99333334 Test accuracy = 0.9878


## Replacing LSTM with GRU cells

In [5]:
gru_cells = [tf.contrib.rnn.GRUCell(num_units=n_neurons)
              for layer in range(n_layers)]

In [6]:
multi_cell = tf.contrib.rnn.MultiRNNCell(gru_cells)

In [7]:
outputs, states = tf.nn.dynamic_rnn(multi_cell, X, dtype=tf.float32)

In [8]:
output = tf.transpose(outputs, [1, 0, 2])
last = tf.gather(output, int(output.get_shape()[0]) - 1)

In [9]:
logits = tf.layers.dense(last, n_outputs, name="softmax")

In [10]:
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy, name="loss")
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

C:\Users\rachi\Anaconda3\lib\site-packages\tensorflow\python\ops\gradients_impl.py:97: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [11]:
init = tf.global_variables_initializer()

In [12]:
mnist = input_data.read_data_sets("/tmp/data/")
X_test = mnist.test.images.reshape((-1, n_steps, n_inputs))
y_test = mnist.test.labels

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [13]:
n_epochs = 10
batch_size = 150
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            X_batch = X_batch.reshape((batch_size, n_steps, n_inputs))
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print("Epoch", epoch, "Train accuracy =", acc_train, "Test accuracy =", acc_test)

Epoch 0 Train accuracy = 0.98 Test accuracy = 0.9676
Epoch 1 Train accuracy = 0.96666664 Test accuracy = 0.9773
Epoch 2 Train accuracy = 0.97333336 Test accuracy = 0.9856
Epoch 3 Train accuracy = 0.99333334 Test accuracy = 0.9856
Epoch 4 Train accuracy = 0.9866667 Test accuracy = 0.9853
Epoch 5 Train accuracy = 0.99333334 Test accuracy = 0.9859
Epoch 6 Train accuracy = 0.9866667 Test accuracy = 0.9862
Epoch 7 Train accuracy = 1.0 Test accuracy = 0.9883
Epoch 8 Train accuracy = 1.0 Test accuracy = 0.9882
Epoch 9 Train accuracy = 0.99333334 Test accuracy = 0.9887
